In [1]:
from openfermion.chem import MolecularData
from openfermionpyscf import run_pyscf
from mindquantum.core.gates import X
from mindquantum.core.circuit import Circuit
from mindquantum.core.operators import Hamiltonian
from mindquantum.simulator import Simulator
from mindquantum.algorithm.nisq import generate_uccsd
import mindspore as ms

ms.set_context(mode=ms.PYNATIVE_MODE, device_target="CPU")

In [2]:
dist = 1.5
geometry = [
    ["Li", [0.0, 0.0, 0.0 * dist]],
    ["H", [0.0, 0.0, 1.0 * dist]],
]
basis = "sto3g"
spin = 0
print("Geometry: \n", geometry)

Geometry: 
 [['Li', [0.0, 0.0, 0.0]], ['H', [0.0, 0.0, 1.5]]]


In [3]:
molecule_of = MolecularData(
    geometry,
    basis,
    multiplicity=2 * spin + 1,
    data_directory='./'
)
molecule_of = run_pyscf(
    molecule_of,
    run_scf=1,
    run_ccsd=1,
    run_fci=1
)

print("Hartree-Fock energy: %20.16f Ha" % (molecule_of.hf_energy))
print("CCSD energy: %20.16f Ha" % (molecule_of.ccsd_energy))
print("FCI energy: %20.16f Ha" % (molecule_of.fci_energy))

Hartree-Fock energy:  -7.8633576215351182 Ha
CCSD energy:  -7.8823529091526794 Ha
FCI energy:  -7.8823622867987284 Ha


In [4]:
molecule_of.save()
molecule_file = molecule_of.filename
print(molecule_file.split('/')[-1])

H1-Li1_sto3g_singlet


In [5]:
hartreefock_wfn_circuit = Circuit([X.on(i) for i in range(molecule_of.n_electrons)])
print(hartreefock_wfn_circuit)

      ┏━━━┓   
q0: ──┨╺╋╸┠───
      ┗━━━┛   
      ┏━━━┓   
q1: ──┨╺╋╸┠───
      ┗━━━┛   
      ┏━━━┓   
q2: ──┨╺╋╸┠───
      ┗━━━┛   
      ┏━━━┓   
q3: ──┨╺╋╸┠───
      ┗━━━┛   


In [6]:
ansatz_circuit, \
init_amplitudes, \
ansatz_parameter_names, \
hamiltonian_QubitOp, \
n_qubits, n_electrons = generate_uccsd(molecule_file, threshold=-1)

ccsd:-7.882352909152679.
fci:-7.882362286798728.


In [7]:
total_circuit = hartreefock_wfn_circuit + ansatz_circuit
total_circuit.summary()
print("Number of parameters: %d" % (len(ansatz_parameter_names)))

                            Circuit Summary                            
╭──────────────────────┬──────────────────────────────────────────────╮
│ Info                 │ value                                        │
├──────────────────────┼──────────────────────────────────────────────┤
│ Number of qubit      │ 12                                           │
├──────────────────────┼──────────────────────────────────────────────┤
│ Total number of gate │ 12612                                        │
│ Barrier              │ 2560                                         │
│ Noise Channel        │ 0                                            │
│ Measurement          │ 0                                            │
├──────────────────────┼──────────────────────────────────────────────┤
│ Parameter gate       │ 640                                          │
│ 44 ansatz parameters │ p0, p8, p1, p9, p2, p10, p3, p11, p4, p12... │
╰──────────────────────┴──────────────────────────────────────────────╯

Number of parameters: 44


In [8]:
sim = Simulator('mqvector', total_circuit.n_qubits)
molecule_pqc = sim.get_expectation_with_grad(Hamiltonian(hamiltonian_QubitOp), total_circuit)

In [9]:
import numpy as np

n_params = len(total_circuit.params_name)
p0 = np.zeros(n_params)
f, g = molecule_pqc(p0)
print("Energy: ", f, "\nshape: ", f.shape, '\n')
print("Gradient: ", g, "\nshape: ", g.shape)

Energy:  [[-7.86335762+0.j]] 
shape:  (1, 1) 

Gradient:  [[[-5.76761758e-11+0.j -8.60518134e-02+0.j  1.19275904e-08+0.j
   -4.85545124e-02+0.j -1.24342308e-14+0.j -3.92769093e-02+0.j
   -3.28108247e-15+0.j -9.59481745e-02+0.j  1.24424913e-14+0.j
   -3.92769093e-02+0.j -3.82408561e-15+0.j -9.59481745e-02+0.j
   -8.60105773e-11+0.j -2.89649669e-02+0.j  2.97699096e-10+0.j
   -4.91813233e-01+0.j -9.35293242e-04+0.j -1.63755737e-16+0.j
   -3.35024598e-17+0.j  1.55629243e-17+0.j  1.59462515e-16+0.j
    2.67655514e-17+0.j -2.16577662e-17+0.j  5.06813117e-03+0.j
    1.08542346e-02+0.j -1.28614265e-02+0.j -4.90653908e-17+0.j
    1.71728860e-17+0.j  1.33973787e-01+0.j -3.03063680e-02+0.j
   -3.37324552e-18+0.j  9.19976050e-19+0.j -1.05649122e-28+0.j
    5.33586109e-17+0.j -3.68357159e-17+0.j  3.33490478e-17+0.j
    1.09202635e-28+0.j  1.15391965e-16+0.j -3.03063680e-02+0.j
   -5.27452935e-17+0.j  4.81071347e-17+0.j -4.22508288e-17+0.j
   -2.44510199e-17+0.j -1.68035039e-03+0.j]]] 
shape:  (1, 1

In [10]:
def fun(p0, molecule_pqc, energy_list=None):
    f, g = molecule_pqc(p0)
    f = np.real(f)[0, 0]
    g = np.real(g)[0, 0]
    if energy_list is not None:
        energy_list.append(f)
        if len(energy_list) % 5 == 0:
            print(f"Step: {len(energy_list)},\tenergy: {f}")
    return f, g

fun(p0, molecule_pqc)

(-7.863357621536964,
 array([-5.76761758e-11, -8.60518134e-02,  1.19275904e-08, -4.85545124e-02,
        -1.24342308e-14, -3.92769093e-02, -3.28108247e-15, -9.59481745e-02,
         1.24424913e-14, -3.92769093e-02, -3.82408561e-15, -9.59481745e-02,
        -8.60105773e-11, -2.89649669e-02,  2.97699096e-10, -4.91813233e-01,
        -9.35293242e-04, -1.63755737e-16, -3.35024598e-17,  1.55629243e-17,
         1.59462515e-16,  2.67655514e-17, -2.16577662e-17,  5.06813117e-03,
         1.08542346e-02, -1.28614265e-02, -4.90653908e-17,  1.71728860e-17,
         1.33973787e-01, -3.03063680e-02, -3.37324552e-18,  9.19976050e-19,
        -1.05649122e-28,  5.33586109e-17, -3.68357159e-17,  3.33490478e-17,
         1.09202635e-28,  1.15391965e-16, -3.03063680e-02, -5.27452935e-17,
         4.81071347e-17, -4.22508288e-17, -2.44510199e-17, -1.68035039e-03]))

In [11]:
from scipy.optimize import minimize

energy_list = []
res = minimize(fun, p0, args=(molecule_pqc, energy_list), method='bfgs', jac=True)

Step: 5,	energy: -7.880227726111826
Step: 10,	energy: -7.8818171240648365
Step: 15,	energy: -7.882213242905283
Step: 20,	energy: -7.8823453369936445
Step: 25,	energy: -7.8823524949903465
Step: 30,	energy: -7.8823526912721755
Step: 35,	energy: -7.882352703403696
Step: 40,	energy: -7.882352708341612


In [12]:
print(f"Ground state: \n{res.fun}\n")
print(f"FCI: \n-7.882362286798721\n")
print(f"Optimized amplitudes: \n{res.x}")

Ground state: 
-7.882352708341612

FCI: 
-7.882362286798721

Optimized amplitudes: 
[ 2.38815317e-04  1.89073105e-03  3.52372251e-02  1.60368559e-02
 -4.30846145e-09  9.09437670e-04 -5.45444468e-10  1.41627870e-02
  3.62569484e-09  9.08701303e-04 -1.05938178e-10  1.41711693e-02
 -5.47703644e-04  4.26818370e-04  2.87168405e-03  5.38109720e-02
  2.34920831e-04 -9.88498886e-08 -2.51432020e-07  2.54936335e-07
 -2.65409411e-08  4.64583011e-08 -4.42267421e-08  1.32809550e-05
 -1.04167881e-04  7.98983208e-04  5.39567507e-10 -2.28346700e-10
 -5.50005419e-02  3.09112487e-03 -4.88778663e-10 -4.89650670e-09
  4.92879467e-09  1.13389943e-07  4.45780819e-09 -1.85315776e-08
 -1.42394064e-12  4.57319777e-10  3.09100040e-03  5.36042785e-08
 -3.87192856e-09  9.99056234e-10  1.51453640e-10  3.72805204e-04]


In [13]:
from mindquantum.algorithm.nisq import Transform
from mindquantum.algorithm.nisq import get_qubit_hamiltonian
from mindquantum.algorithm.nisq import uccsd_singlet_generator, uccsd_singlet_get_packed_amplitudes
from mindquantum.core.operators import TimeEvolution

In [14]:
hamiltonian_QubitOp = get_qubit_hamiltonian(molecule_of)

In [15]:
ucc_fermion_ops = uccsd_singlet_generator(
    molecule_of.n_qubits, molecule_of.n_electrons, anti_hermitian=True)

In [16]:
ucc_qubit_ops = Transform(ucc_fermion_ops).jordan_wigner()

In [17]:
ansatz_circuit = TimeEvolution(ucc_qubit_ops.imag, 1.0).circuit
ansatz_parameter_names = ansatz_circuit.params_name

In [18]:
total_circuit = hartreefock_wfn_circuit + ansatz_circuit
total_circuit.summary()

                                  Circuit Summary                                  
╭──────────────────────┬──────────────────────────────────────────────────────────╮
│ Info                 │ value                                                    │
├──────────────────────┼──────────────────────────────────────────────────────────┤
│ Number of qubit      │ 12                                                       │
├──────────────────────┼──────────────────────────────────────────────────────────┤
│ Total number of gate │ 12612                                                    │
│ Barrier              │ 2560                                                     │
│ Noise Channel        │ 0                                                        │
│ Measurement          │ 0                                                        │
├──────────────────────┼──────────────────────────────────────────────────────────┤
│ Parameter gate       │ 640                                                      │
│ 44 ansatz parameters │ s_0, d1_0, s_1, d1_1, s_2, d1_2, s_3, d1_3, s_4, d1_4... │
╰──────────────────────┴──────────────────────────────────────────────────────────╯

In [19]:
init_amplitudes_ccsd = uccsd_singlet_get_packed_amplitudes(
    molecule_of.ccsd_single_amps, molecule_of.ccsd_double_amps, molecule_of.n_qubits, molecule_of.n_electrons)
init_amplitudes_ccsd = [init_amplitudes_ccsd[param_i] for param_i in ansatz_parameter_names]

In [20]:
grad_ops = Simulator('mqvector', total_circuit.n_qubits).get_expectation_with_grad(
    Hamiltonian(hamiltonian_QubitOp.real),
    total_circuit)

In [21]:
energy_list = []
res = minimize(fun, init_amplitudes_ccsd, args=(grad_ops, energy_list), method='bfgs', jac=True)

Step: 5,	energy: -7.878223283110822
Step: 10,	energy: -7.880288481802435
Step: 15,	energy: -7.8820356683191415
Step: 20,	energy: -7.882302370884269
Step: 25,	energy: -7.882349803535784
Step: 30,	energy: -7.882352702053716
Step: 35,	energy: -7.88235270792034
Step: 40,	energy: -7.882352708346457


The final optimized result is shown as below.

In [22]:
print(f"Ground state: \n{res.fun}\n")
print(f"FCI: \n-7.882362286798721\n")
print(f"Optimized amplitudes: \n{res.x}")

Ground state: 
-7.882352708346457

FCI: 
-7.882362286798721

Optimized amplitudes: 
[-2.38636816e-04  1.89071890e-03 -3.52372349e-02  1.60368129e-02
  1.34665623e-08  9.09430837e-04 -4.17115460e-10  1.41641317e-02
 -1.28144111e-08  9.08694024e-04  4.20204892e-10  1.41698036e-02
  5.47710797e-04  4.26820871e-04 -2.87169433e-03  5.38109417e-02
  2.34627619e-04  4.32417990e-07 -1.26325933e-08 -1.39715964e-08
 -4.27213491e-07  1.24814811e-08  1.37984739e-08  1.32946010e-05
  7.99032002e-04 -1.04121599e-04  1.25575236e-09 -1.24558942e-09
 -5.50005457e-02  3.09142613e-03  1.53511697e-08 -3.40030804e-08
  3.29912584e-08  3.67660373e-07 -2.59584687e-08  2.22720149e-10
  4.47254154e-11  1.06718989e-09  3.09086800e-03 -3.54929776e-07
  1.95048358e-08  5.27817957e-09 -1.03614664e-09  3.72830373e-04]


In [23]:
from mindquantum.utils.show_info import InfoTable

InfoTable('mindquantum', 'scipy', 'numpy')

Software,Version
mindquantum,0.9.11
scipy,1.10.1
numpy,1.23.5
System,Info
Python,3.9.16
OS,Linux x86_64
Memory,8.3 GB
CPU Max Thread,8
Date,Mon Jan 1 00:25:57 2024
